# Chat Prompt Templates

The Cassandra-specific approach can be seamlessly integrated
with LangChain's "chat prompt templates".

In [1]:
from langchain.prompts import createCassandraPromptTemplate

In [2]:
from cqlsession import getCQLSession, getCQLKeyspace
astraSession = getCQLSession()
astraKeyspace = getCQLKeyspace()

/home/stefano/.virtualenvs/langchain-cassio-3.10/lib/python3.10/site-packages/cassandra/io/asyncorereactor.py:34: DeprecationWarning: The asyncore module is deprecated and will be removed in Python 3.12. The recommended replacement is asyncio
  import asyncore
/home/stefano/.virtualenvs/langchain-cassio-3.10/lib/python3.10/site-packages/cassandra/datastax/cloud/__init__.py:173: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  ssl_context = SSLContext(PROTOCOL_TLS)
/home/stefano/.virtualenvs/langchain-cassio-3.10/lib/python3.10/site-packages/cassandra/io/asyncorereactor.py:347: DeprecationWarning: ssl.match_hostname() is deprecated
  self._connect_socket()


This is the prompt for a single message in the chat sequence.

We create it similarly as for a "stand-alone Cassandra prompt template".

In [3]:
systemTemplate = """
You are a chat assistant, helping a user of age {user_age} from a city
they refer to as {city_nickname}.
"""

In [4]:
cassSystemPrompt = createCassandraPromptTemplate(
    session=astraSession,
    keyspace=astraKeyspace,
    template=systemTemplate,
    input_variables=['city', 'name'],
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
)

In [5]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
systemMessagePrompt = SystemMessagePromptTemplate(prompt=cassSystemPrompt)

## A sequence of messages

Once we wrapped a single prompt template as a "system message prompt", let's make it part of a longer chat conversation:

In [6]:
humanTemplate = "{text}"
humanMessagePrompt = HumanMessagePromptTemplate.from_template(humanTemplate)

In [7]:
cassChatPrompt = ChatPromptTemplate.from_messages(
    [systemMessagePrompt, humanMessagePrompt]
)

### Rendering

LangChain takes care of correctly propagating the rendering steps throughout the sequence of messages, including the Cassandra-backed template:

In [8]:
print(cassChatPrompt.format_prompt(
    city='turin',
    name='beppe',
    text='Assistant, please help me!'
).to_string())

System: 
You are a chat assistant, helping a user of age 2 from a city
they refer to as CereaNeh.

Human: Assistant, please help me!
